In [7]:
import sys
!{sys.executable} -m pip install mediapipe opencv-python

In [8]:
import cv2
import mediapipe as mp
import math

# 初始化 MediaPipe 手部偵測工具
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

print("模組導入成功！")

模組導入成功！


In [9]:
# 計算兩個向量之間的角度

def vector_2d_angle(v1, v2):
    """
    根據兩點的座標，計算角度
    
    參數:
        v1: 第一個向量 (x, y)
        v2: 第二個向量 (x, y)

    返回:
        angle: 兩向量之間的角度（度數）
    """
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]

    try:
        # 使用餘弦定理計算角度
        angle_ = math.degrees(
            math.acos(
                (v1_x*v2_x + v1_y*v2_y) / 
                (((v1_x**2 + v1_y**2)**0.5) * ((v2_x**2 + v2_y**2)**0.5))
            )
        )
    except:
        angle_ = 180

    return angle_

print("vector_2d_angle 函數定義完成！")

vector_2d_angle 函數定義完成！


In [10]:
# 計算手指角度

def hand_angle(hand_):
    """
    根據傳入的 21 個手部節點座標，計算五根手指的角度
    
    參數:
        hand_: 包含 21 個手部關鍵點的座標列表
    
    返回:
        angle_list: 五根手指的角度列表 [大拇指, 食指, 中指, 無名指, 小拇指]
    """
    angle_list = []
    
    # 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[2][0])), (int(hand_[0][1]) - int(hand_[2][1]))),
        ((int(hand_[3][0]) - int(hand_[4][0])), (int(hand_[3][1]) - int(hand_[4][1])))
    )
    angle_list.append(angle_)
    
    # 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[6][0])), (int(hand_[0][1]) - int(hand_[6][1]))),
        ((int(hand_[7][0]) - int(hand_[8][0])), (int(hand_[7][1]) - int(hand_[8][1])))
    )
    angle_list.append(angle_)
    
    # 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[10][0])), (int(hand_[0][1]) - int(hand_[10][1]))),
        ((int(hand_[11][0]) - int(hand_[12][0])), (int(hand_[11][1]) - int(hand_[12][1])))
    )
    angle_list.append(angle_)
    
    # 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[14][0])), (int(hand_[0][1]) - int(hand_[14][1]))),
        ((int(hand_[15][0]) - int(hand_[16][0])), (int(hand_[15][1]) - int(hand_[16][1])))
    )
    angle_list.append(angle_)
    
    # 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[18][0])), (int(hand_[0][1]) - int(hand_[18][1]))),
        ((int(hand_[19][0]) - int(hand_[20][0])), (int(hand_[19][1]) - int(hand_[20][1])))
    )
    angle_list.append(angle_)
    
    return angle_list

print("hand_angle 函數定義完成！")

hand_angle 函數定義完成！


In [11]:
# 手勢識別

def hand_pos(finger_angle, finger_points=None):
    """
    根據手指角度的列表內容，返回對應的手勢名稱

    參數:
        finger_angle: 五根手指的角度列表
        finger_points: 可選，21 個手部關鍵點座標，用於判斷朝向（例如倒讚判斷大拇指朝上或朝下）

    返回:
        str: 手勢名稱

    角度規則: 小於 50 表示手指伸直，大於等於 50 表示手指捲縮
    """
    f1 = finger_angle[0]   # 大拇指角度
    f2 = finger_angle[1]   # 食指角度
    f3 = finger_angle[2]   # 中指角度
    f4 = finger_angle[3]   # 無名指角度
    f5 = finger_angle[4]   # 小拇指角度

    # 計算手部 bounding box 高度（用於正規化方向判斷）
    box_h = None
    if finger_points is not None and len(finger_points) > 0:
        ys = [p[1] for p in finger_points]
        box_h = max(ys) - min(ys)

    # 新增：同時比大拇指 + 中指 + 小指（三指同時伸直），視為特殊不雅手勢
    # 條件：大拇指、中指、小指 伸直 (角度 < 50)，且食指、無名指 捲縮 (角度 >=50)
    if f1<50 and f2>=50 and f3<50 and f4>=50 and f5<50:
        return 'thumb_mid_pinky'

    # 判斷讚 / 倒讚：大拇指伸直且其他手指捲縮
    if f1<50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        # 若有座標資料，使用大拇指指尖與大拇指 MCP 的 y 差值（相對於手部高度）判斷朝向
        if finger_points is not None and len(finger_points) > 4:
            # 使用 thumb_mcp (index 2) 與 thumb_tip (index 4)
            thumb_mcp = finger_points[2]
            thumb_tip = finger_points[4]
            dy = thumb_tip[1] - thumb_mcp[1]
            # 使用相對手部高度作為閾值，避免不同距離造成差異
            threshold = (box_h * 0.12) if box_h and box_h>0 else 10
            if dy > threshold:
                return 'bad!!!'   # 指尖比 MCP 更往下 -> 倒讚
            elif dy < -threshold:
                return 'good'      # 指尖比 MCP 更往上 -> 讚
            else:
                # 不明顯的情況，回傳 'good'（較保守）
                return 'good'
        else:
            # 沒有座標資訊時，保守回傳 'good'
            return 'good'

    # 中指 (no) 判定
    if f1>=50 and f2>=50 and f3<50 and f4>=50 and f5>=50:
        return 'no!!!'       # 比中指（會加馬賽克）
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5<50:
        return 'ROCK!'       # 搖滾
    elif f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return '0'           # 拳頭
    elif f1>=50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '1'           # 1
    elif f1>=50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '2'           # 2
    elif f1>=50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'          # OK
    elif f1<50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'          # OK (另一種)
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5>50:
        return '3'           # 3
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '4'           # 4
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '5'           # 5
    elif f1<50 and f2>=50 and f3>=50 and f4>=50 and f5<50:
        return '6'           # 6
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '7'           # 7
    elif f1<50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '8'           # 8
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5>=50:
        return '9'           # 9

    else:
        return ''            # 無法識別

print("hand_pos 函數定義完成！")

hand_pos 函數定義完成！


In [12]:
# 啟動手勢識別

import numpy as np

# 初始化攝影機
cap = cv2.VideoCapture(0)
fontFace = cv2.FONT_HERSHEY_SIMPLEX  # 文字字型
lineType = cv2.LINE_AA               # 文字邊框

print("正在啟動攝影機...")
print("按 'q' 鍵退出程式")

# 啟用 MediaPipe 手部偵測
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("無法開啟攝影機")
        exit()
    
    w, h = 720, 540  # 影像尺寸

    # 用於 frame-to-frame 平滑的先前 bounding box
    prev_bbox = None
    alpha = 0.65  # 平滑係數（越接近 1 越穩定但延遲越大）

    # 多幀確認（debounce）狀態
    gesture_buffer_text = ''
    gesture_buffer_count = 0
    debounce_frames = 3  # 要求連續出現多少幀才觸發馬賽克

    while True:
        ret, img = cap.read()
        img = cv2.resize(img, (w, h))  # 縮小尺寸，加快處理效率
        
        if not ret:
            print("無法讀取影像")
            break
        
        # 轉換成 RGB 色彩供 MediaPipe 處理
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img2)
        
        # 如果偵測到手部
        if results.multi_hand_landmarks:
            detections = []  # 暫存每隻手的判斷結果，稍後依 debounce 決定是否馬賽克

            for hand_landmarks in results.multi_hand_landmarks:
                finger_points = []  # 記錄手指節點座標
                fx = []             # 記錄所有 x 座標
                fy = []             # 記錄所有 y 座標
                
                # 取得 21 個手部關鍵點
                for i in hand_landmarks.landmark:
                    x = int(i.x * w)
                    y = int(i.y * h)
                    finger_points.append((x, y))
                    fx.append(x)
                    fy.append(y)

                if finger_points:
                    # 計算手指角度
                    finger_angle = hand_angle(finger_points)
                    
                    # 識別手勢（傳入座標以便判斷方向）
                    text = hand_pos(finger_angle, finger_points)

                    detections.append({
                        'text': text,
                        'finger_points': finger_points,
                        'fx': fx,
                        'fy': fy,
                    })

            # 決定本幀的 candidate（若有多隻手發現不雅手勢，取出現次數最多的那個）
            blacklist = ('no!!!', 'bad!!!', 'thumb_mid_pinky', 'ok')
            frame_candidates = [d['text'] for d in detections if d['text'] in blacklist]
            if frame_candidates:
                # 選擇出現最多的 label
                candidate = max(set(frame_candidates), key=frame_candidates.count)
                if candidate == gesture_buffer_text:
                    gesture_buffer_count += 1
                else:
                    gesture_buffer_text = candidate
                    gesture_buffer_count = 1
            else:
                gesture_buffer_text = ''
                gesture_buffer_count = 0

            # 根據 debounce 結果決定對每個偵測進行馬賽克或僅顯示文字
            for d in detections:
                text = d['text']
                finger_points = d['finger_points']
                fx = d['fx']
                fy = d['fy']

                # 是否應該馬賽克：必須為 blacklist 且等於 buffer 且連續幀數已達閾值
                should_mosaic = (text in blacklist and text == gesture_buffer_text and gesture_buffer_count >= debounce_frames)

                if should_mosaic:
                    pts = np.array(finger_points, dtype=np.int32)
                    try:
                        hull = cv2.convexHull(pts)
                        x, y, w_box, h_box = cv2.boundingRect(hull)
                    except Exception:
                        # fallback to min/max if convexHull fails
                        x_min = min(fx)
                        x_max = max(fx)
                        y_min = min(fy)
                        y_max = max(fy)
                        x, y, w_box, h_box = x_min, y_min, x_max - x_min, y_max - y_min

                    # padding 與最小尺寸
                    pad = int(max(w_box, h_box) * 0.20) + 8
                    x_min = x - pad
                    y_min = y - pad
                    x_max = x + w_box + pad
                    y_max = y + h_box + pad

                    # 最小尺寸保護
                    min_dim = 50
                    if (x_max - x_min) < min_dim:
                        cx = (x_min + x_max) // 2
                        x_min = cx - min_dim // 2
                        x_max = cx + min_dim // 2
                    if (y_max - y_min) < min_dim:
                        cy = (y_min + y_max) // 2
                        y_min = cy - min_dim // 2
                        y_max = cy + min_dim // 2

                    # 邊界檢查
                    if x_max > w: x_max = w
                    if y_max > h: y_max = h
                    if x_min < 0: x_min = 0
                    if y_min < 0: y_min = 0

                    # frame-to-frame 平滑
                    if prev_bbox is not None:
                        px1, py1, px2, py2 = prev_bbox
                        x_min = int(px1 * alpha + x_min * (1 - alpha))
                        y_min = int(py1 * alpha + y_min * (1 - alpha))
                        x_max = int(px2 * alpha + x_max * (1 - alpha))
                        y_max = int(py2 * alpha + y_max * (1 - alpha))

                    prev_bbox = (x_min, y_min, x_max, y_max)

                    # 製作馬賽克效果（確保區域有效）
                    if x_max > x_min and y_max > y_min:
                        mosaic_w = x_max - x_min
                        mosaic_h = y_max - y_min
                        mosaic = img[y_min:y_max, x_min:x_max]
                        # 若區域太小，略微放大取樣以免變形
                        down_w, down_h = max(8, min(16, mosaic_w // 4)), max(8, min(16, mosaic_h // 4))
                        try:
                            mosaic = cv2.resize(mosaic, (down_w, down_h), interpolation=cv2.INTER_LINEAR)
                            mosaic = cv2.resize(mosaic, (mosaic_w, mosaic_h), interpolation=cv2.INTER_NEAREST)
                            img[y_min:y_max, x_min:x_max] = mosaic
                        except Exception:
                            pass

                    # 不論是哪一種被馬賽克的手勢，都顯示字幕 'BAD!!!'（紅色）並加底色以提高可讀性
                    txt = 'BAD!!!'

                    # 將文字置於馬賽克上方，如空間不足則放在內部；並加黑色底色提高可讀性
                    tx = x_min
                    ty = y_min - 10
                    (tw, th), _ = cv2.getTextSize(txt, fontFace, 1.2, 4)
                    box_x1 = max(0, tx - 6)
                    box_y1 = max(0, ty - th - 6)
                    box_x2 = min(w, tx + tw + 6)
                    box_y2 = min(h, ty + 6)
                    cv2.rectangle(img, (box_x1, box_y1), (box_x2, box_y2), (0,0,0), -1)
                    cv2.putText(img, txt, (tx, ty), fontFace, 1.2, (0, 0, 255), 4, lineType)
                else:
                    # 尚未達 debounce 閾值，或並非 blacklist，顯示手勢文字（或不顯示）
                    if text:
                        cv2.putText(img, text, (30, 120), fontFace, 5, (255, 255, 255), 10, lineType)

        # 顯示影像
        cv2.imshow('Hand Gesture Recognition', img)
        
        # 按 'q' 退出
        if cv2.waitKey(5) == ord('q'):
            print("\n程式結束")
            break

cap.release()
cv2.destroyAllWindows()
print("攝影機已關閉")

正在啟動攝影機...
按 'q' 鍵退出程式

程式結束
攝影機已關閉
